In [1]:
from mido import MidiFile

In [65]:
## Importa la tabla que parsea numeros de notas a nombre y octava
import pandas as pd
import numpy as np
import plotly.express as px
midi_notes = pd.read_csv('midi_notes.csv',sep=";")
midi_scales_chords = pd.read_csv('scales.csv',sep=";")

In [73]:
## Levanta el archivo midi, devuelve cantidad de tracks y mensajes contenidos
def load_midi_file(files_mid):
    mid = MidiFile(files_mid+ '.mid', clip=False)
    return mid


mid = load_midi_file('4bros')
print(mid)
print('ticks por beat:', mid.ticks_per_beat)
duracion_tema = mid.length
print('segundos:', duracion_tema)


<midi file '4bros.mid' type 0, 1 tracks, 5773 messages>
ticks por beat: 96
segundos: 96.2228020312491


In [102]:
## Itera sobre los mensaje midi de los tracks con mensjaes y genera un dataframe con notas, velocity, tick_start tick_stop
def get_theme_df(mid):
    dict_notes = {}
    dict_notes_end = {}
    dict_active_notes = {}
    count_notes = 0
    count_notes_end = 0
    last_note_on = 0
    info = []
    controls = []
    key_signatures = []
    time_signatures = []
    instrument_track = []
    n_track = 0
    n_tracks_used = 0
    for track in mid.tracks:

        track_name = track.name + str(n_track)
        info.append(track_name)
        n_track = n_track + 1 
        if len(track) > 100:      
            n_tracks_used = n_tracks_used + 1
            print('mensajes:',len(track))
        time = 0
        has_note_off = any(msg.type == 'note_off' for msg in track)
        for msg in track:          
            if msg.type in ['note_on', 'note_off']:
                time = time + msg.time
                if (has_note_off and (msg.type == 'note_on')) or (not has_note_off and msg.velocity > 0):
                    dict_notes[count_notes] = {"note_num": msg.note, "start": time, "velocity": msg.velocity, "time_dif": time - last_note_on, "track_name": track_name}
                    dict_active_notes[msg.note] = time
                    count_notes = count_notes + 1
                    last_note_on = time
                else:
                    dict_notes_end[count_notes_end] = {"note_num": msg.note,"track_name": track_name, "start": dict_active_notes[msg.note], "end": time}
                    count_notes_end = count_notes_end + 1
            else:
                if (msg.type == 'control_change'):
                    controls.append(msg.value)
                elif (msg.type == 'key_signature'):
                    key_signatures.append(msg.key)
                elif (msg.type == 'time_signature'):
                    time_signatures.append(str(msg.numerator) + '/' + str(msg.denominator))
                elif (msg.type == 'program_change'):
                    instrument_track.append(str(n_track) + "-" + str(msg))
    tema_df = pd.DataFrame.from_dict(dict_notes, "index")
    tema_df_notes_end = pd.DataFrame.from_dict(dict_notes_end, "index")
    tema_df_merged = pd.merge(tema_df, tema_df_notes_end,on=['note_num','start','track_name'])
    controls = pd.Series(controls).head(40)
    controls = controls[controls > 10].sum() / n_tracks_used
    return tema_df_merged, info, controls, key_signatures, time_signatures, n_tracks_used, instrument_track                 

In [10]:
## Reemplaza outlyers de duración
def limit_outlyer_duration_notes():
    notes_weight = pd.cut(tema_df.duration, 6)
    print('duration notes before:')
    display(notes_weight.value_counts())

    outlyeras_duration = pd.DataFrame(tema_df.duration.quantile([0.05,0.95]))

    mask_outlyers_lower = tema_df.duration < outlyeras_duration.duration[0.05]
    print('lowers modificados:',mask_outlyers_lower.sum(), 'val:', outlyeras_duration.duration[0.05])
    tema_df.loc[mask_outlyers_lower,'duration'] = outlyeras_duration.duration[0.05]

    mask_outlyers_higher = tema_df.duration > outlyeras_duration.duration[0.95]
    print('highers modificados:', mask_outlyers_higher.sum(), 'val:', outlyeras_duration.duration[0.95])
    tema_df.loc[mask_outlyers_higher,'duration'] = outlyeras_duration.duration[0.95]

    notes_weight = pd.cut(tema_df.duration, 6)
    print('duration notes after:')
    display(notes_weight.value_counts().head(7))

In [22]:
# Crea una mascara para limitar el ploteo en el tiempo
def plot_theme(data_to_plot, plot_from_second=0, plot_to_second=30, size_plot='duration', color_plot='grupos_track'):
    ## Poltea las notas en el tiempo
    mask_tiempo_from = (tema_df.segundo >= plot_from_second)
    mask_tiempo_to =  (tema_df.segundo <= plot_to_second)
    fig = px.scatter(data_frame = tema_df[mask_tiempo_from & mask_tiempo_to], x = "segundo", y = "note_num",
                     size = size_plot, color = color_plot, opacity = 1, color_discrete_sequence=None, labels='note', height=600,)

    fig.show()

In [11]:
## Reemplaza los tiempos de notas muy cercanas por notas simultaneas
def cuantize():
    tema_df2_mask = (tema_df.iloc[1:len(tema_df),:].note_simultaneous == True)
    tema_df2_mask = tema_df2_mask
    tema_df2_mask[0] = False

    tema_df.loc[tema_df2_mask, 'start'] = np.NaN
    tema_df.start = tema_df.start.fillna(method='ffill')
    tema_df.start = tema_df.start.astype(int)

In [20]:
def restore_df():
    tema_df = tema_df_copy

In [103]:
## Instancia el archivo midi
tema_df, info, controls, key_signatures, time_signatures, n_tracks_used, instrument_track = get_theme_df(mid)  

mensajes: 5773


In [67]:
track_n = 0
for track in mid.tracks:   
    print('track:',track)
    for msg in track:   
        if msg.type not in ['note_on', 'note_off', 'control_change', 'pitchwheel', 'midi_port', 'end_of_track']:
            print(msg)
    track_n = track_n + 1

track: <midi track '4Brothrs' 5773 messages>
<meta message track_name name='4Brothrs' time=0>
<meta message set_tempo tempo=289855 time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
program_change channel=0 program=56 time=1
program_change channel=1 program=58 time=0
program_change channel=2 program=32 time=0
program_change channel=3 program=65 time=0
program_change channel=4 program=66 time=0
program_change channel=5 program=66 time=0
program_change channel=6 program=67 time=0
program_change channel=7 program=71 time=0
program_change channel=9 program=0 time=0


In [74]:
for msg in mid.play():
    port.send(msg)

NameError: name 'port' is not defined

In [68]:
mid.print_tracks(meta_only=True)

=== Track 0
<meta message track_name name='4Brothrs' time=0>
<meta message set_tempo tempo=289855 time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message end_of_track time=0>


In [21]:
## Instancia el archivo midi
tema_df, info, controls, key_signatures, time_signatures, n_tracks_used, instrument_track = get_theme_df(mid)  

tema_df_copy = tema_df.copy()
## Calcula la duración de cada nota
tema_df.loc[:,'duration'] = tema_df.end - tema_df.start

## Parsea a enteros los valores numéricos
for col in tema_df.loc[:,tema_df.columns!="track_name"].columns:
    tema_df[col] = pd.to_numeric(tema_df[col])
    
## agrega el nombre y octava de notas a la tabla
tema_df = pd.merge(tema_df, midi_notes,how='left',left_on='note_num',right_on='note_number').drop('note_number',axis=1)

##elimina notas demasiado cortas y demasiado largas que pueden afectar al análisis
limit_outlyer_duration_notes()

## Categoriza duración

cat_duration = pd.cut(tema_df.duration, 6, labels=['xxs','xs','s','m','l','xl'])
print('category duration notes:')
display(cat_duration.value_counts())
tema_df.loc[:,'cat_duration'] = cat_duration

## Categoriza VELOCITY
cat_velocity = pd.cut(tema_df.velocity, 6, labels=['pp','p','m','mf','f','ff'])
print('category velocity notes:')
display(cat_velocity.value_counts())
tema_df.loc[:,'cat_velocity'] = cat_velocity

## Describe, muchos de estos valores van a ser utiles como predictores
tema_describe = tema_df.describe()

## Cuantificación de notas:
## define cuales notas están tan cera que deberían estar en el mismo instante

unit_duplicate = tema_describe.loc['min','duration'] / 1.5
tema_df.loc[:,'note_simultaneous'] = tema_df.time_dif < unit_duplicate
print('ticks para quantizar: ',unit_duplicate)

## Reemplaza los tiempos de notas muy cercanas por notas simultaneas
cuantize()

## calcula la cantidad de ticks x segundo

ticks_por_seg = tema_df.end.max() / duracion_tema
print('Ticks por segundo:', ticks_por_seg)

## Calcula la cantidad de notas que existen en simultaneo

tema_df_simultaneous =  tema_df.start.value_counts()
tema_df_simultaneous_times = tema_df_simultaneous.loc[tema_df_simultaneous > 1].index.to_list()
print('cant. notas en simultaneo:',len(tema_df_simultaneous_times))

tema_df.note_simultaneous = False
tema_df.loc[:,'note_simultaneous'] = tema_df.start.apply(lambda x: True if x in tema_df_simultaneous_times else False) 

mensajes: 825
mensajes: 925
mensajes: 763
mensajes: 495
mensajes: 839
mensajes: 838
mensajes: 2095
mensajes: 997
mensajes: 521
mensajes: 271
duration notes before:


(25.959, 200.5]    3410
(200.5, 374.0]      799
(374.0, 547.5]       27
(721.0, 894.5]       13
(894.5, 1068.0]       1
(547.5, 721.0]        0
Name: duration, dtype: int64

lowers modificados: 42 val: 54.0
highers modificados: 83 val: 216.0
duration notes after:


(81.0, 108.0]     2746
(189.0, 216.0]     840
(53.838, 81.0]     647
(135.0, 162.0]      17
(162.0, 189.0]       0
(108.0, 135.0]       0
Name: duration, dtype: int64

category duration notes:


xs     2746
xl      840
xxs     647
m        17
l         0
s         0
Name: duration, dtype: int64

category velocity notes:


m     4250
ff       0
f        0
mf       0
p        0
pp       0
Name: velocity, dtype: int64

ticks para quantizar:  36.0
Ticks por segundo: 345.1639625402987
cant. notas en simultaneo: 528


In [23]:
## Cada nota l y x l van a tener mas puntos que despues van a ser removidos

tema_df.loc[:,'note_fake'] = 0
long_notes_mask = tema_df.cat_duration.apply(lambda x: x in ['l','xl'])
fake_notes = tema_df[long_notes_mask]
fake_notes.start = fake_notes.apply(lambda x: x.start + (x.end - x.start) / 2 if x.cat_duration == 'l' else x.start + (x.end - x.start) / 3,axis=1)

fake_notes.duration = fake_notes.duration / 2
fake_notes.note_fake = 1

xlong_notes_mask = tema_df.cat_duration == 'xl'
fake_notes_lx = tema_df[xlong_notes_mask]
fake_notes_lx.start = fake_notes_lx.start + (fake_notes_lx.end - fake_notes_lx.start)  / 3 * 2
fake_notes_lx.duration = fake_notes_lx.duration / 2
fake_notes_lx.note_fake = 1

tema_df = pd.concat([tema_df,fake_notes, fake_notes_lx]).sort_values('start')

## Convierte unidad de medida de timpo Ticks a segundos en cada nota

tema_df.loc[:,'segundo'] = tema_df.start / ticks_por_seg

## Implementacion DBSCAN para encontrar grupos de notas y segmentarlas

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

X = tema_df[['start', 'note_num']]
X.loc[:,'start'] = X.start / 15
X.loc[:,'note_num'] = X.note_num / 0.2

X = X.values
print(X[0:5])

# Implementación de DBSCAN

dbscan = DBSCAN( eps=29, min_samples=3)
y_dbscan = dbscan.fit_predict(X)
y_dbscan

labels = dbscan.labels_
#unique_labels=set(labels);
tema_df.loc[:,'grupos'] = labels
cant_grupos = pd.Series(labels).unique().shape[0]
print('cantidad de grupos:',cant_grupos)

## Concatena los grupos creados con DBSCAN con las cadenas de notas recibidas en cada track

tema_df.loc[:,'grupos_track'] = tema_df.grupos.astype(str) + "_" + tema_df.track_name

plot_theme(tema_df)

C:\Users\Patricio\anaconda3\envs\dhdsblend\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Patricio\anaconda3\envs\dhdsblend\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[[  8. 240.]
 [  8. 360.]
 [  8. 315.]
 [  8. 240.]
 [  8. 180.]]
cantidad de grupos: 8


In [14]:
## elimina notas falsas
notes_fake_mask = tema_df.note_fake == 1
tema_df.drop(tema_df[notes_fake_mask].index, inplace=True)

## Imprime el dataset completo ordenado por grupos descartando el ruido
## Calcula el ruido
print('noise notes: %', round((tema_df.grupos == -1).sum() / tema_df.shape[0],2))
mask_noise = tema_df.grupos != -1
tema_df.sort_values(['grupos_track','start'],ascending=[True,True],inplace=True)

## Shape final del dataset
notas_totales = tema_df.shape[0]


## indice de actividad (cantidad de notas) por tiempo
cant_eventos_individuales = (notas_totales - len(tema_df_simultaneous_times) / 2)
actividad_por_tiempo = cant_eventos_individuales / duracion_tema
velocity_avg = tema_df.cat_velocity.value_counts(normalize=True)
length_notes_avg = tema_df.cat_duration.value_counts(normalize=True)

## Analiza proporciones de notas y duraciones mas repetidas
notes_weight = round(tema_df.note.value_counts(normalize=True) * 100,2)
print('notes most used:')
display(notes_weight.head(5))

notes_weight = round(tema_df.note_name.value_counts(normalize=True) * 100,2)
print('scale most used:')
all_values_notes = pd.DataFrame(notes_weight).reset_index()
most_probable_scale = all_values_notes.head(7)
display(most_probable_scale)
scale_coverage = notes_weight.head(7).sum()
avr_vertical_notes = tema_df.note_simultaneous.sum() / notas_totales
cant_pedal_sustain = controls
cant_eventos_por_pedal = cant_eventos_individuales / cant_pedal_sustain if cant_pedal_sustain > 5 else np.NaN
cant_pedales_seg = cant_pedal_sustain / duracion_tema if cant_pedal_sustain > 5 else np.NaN

#obtiene informacion de la escala
nombre_escala = pd.merge(most_probable_scale, midi_scales_chords, how='left', left_on='index', right_on='note_name')
nombre_escala.fillna(0,inplace=True)
nombre_escala_T = nombre_escala.T
nombre_escala_T
mask = nombre_escala_T.apply(lambda x: True if all(x != 0) else False, axis=1)
mask
tabla_esacla = nombre_escala_T[mask].T

tabla_esacla

nombre_columna_Tmaj = tabla_esacla.columns[3]
tonalidad  = 0
tonalidad_escala = 'M'
dic = {6:1, 7:2, 1:3, 2:4, 3:5, 4:6, 5:7}
if nombre_columna_Tmaj != "U":
    tabla_esacla.set_index(tabla_esacla.columns[3],inplace=True,drop=False)
    mayor_chord_coverage = tabla_esacla.loc[[1,3,5],:'note_name_x'].sum()[1]
    minor_chord_coverage = tabla_esacla.loc[[6,1,3],:'note_name_x'].sum()[1]
    print('mayor chord:',round(mayor_chord_coverage,2),'| minor chord:',round(minor_chord_coverage,2))
    

    if minor_chord_coverage > mayor_chord_coverage:
        tonalidad = tabla_esacla.iloc[0,2]
        # reemplazo el 6to grado por la tónica
         # define desired replacements here

        tabla_esacla[nombre_columna_Tmaj] = tabla_esacla[nombre_columna_Tmaj].apply(lambda x: dic[x])
        tabla_esacla
        tonalidad_escala = 'm'
        tabla_esacla.set_index(nombre_columna_Tmaj,inplace=True,drop=False)
        tabla_esacla.sort_index(inplace=True)
    else:
        tonalidad = nombre_columna_Tmaj

    print(tonalidad + tonalidad_escala) 
else:
    if 'b' in key_signatures[0]:
        dict_keys = {'Db':'C#', 'Eb':'D#', 'Gb':'F#', 'Ab':'G#', 'Bb':'A#'}
        tonalidad =  dict_keys[key_signatures[0]]
    else:
        tonalidad = key_signatures[0]

    midi_scales_chords_weighted = pd.merge(midi_scales_chords[['note_name', tonalidad]], all_values_notes, how='left', left_on='note_name', right_on='index',)
    midi_scales_chords_weighted.drop('index',axis=1,inplace=True)
    midi_scales_chords_weighted.columns = ['note_name', 'scale', 'weight']
    midi_scales_chords_weighted.set_index(midi_scales_chords_weighted.columns[1],inplace=True,drop=False)
    mayor_chord_coverage = midi_scales_chords_weighted.loc[[1,3,5],'weight'].sum()
    minor_chord_coverage = midi_scales_chords_weighted.loc[[6,1,3],'weight'].sum()
    print('mayor chord:',round(mayor_chord_coverage,2),'| minor chord:',round(minor_chord_coverage,2))
    if minor_chord_coverage > mayor_chord_coverage:
        tonalidad_escala = 'm'
        midi_scales_chords_weighted_mask = midi_scales_chords_weighted.scale > 0
        midi_scales_chords_weighted.loc[midi_scales_chords_weighted_mask, 'scale'] = midi_scales_chords_weighted.loc[midi_scales_chords_weighted_mask, 'scale'].apply(lambda x: dic[x])
    midi_scales_chords_weighted.sort_index(inplace=True)
    midi_scales_chords_weighted.drop('scale',axis=1,inplace=True)
    scale_coverage = midi_scales_chords_weighted.head(7).sum()[1]
    tabla_esacla = midi_scales_chords_weighted
    tabla_esacla
    

## crea el Music stats    
music_stats = pd.DataFrame(columns=['tonalidad','tonalidad_escala','scale_coverage','mayor_chord_coverage','minor_chord_coverage',
                              'sc_1','sc_2','sc_3','sc_4','sc_5','sc_6','sc_7', 'avr_vertical_notes','cant_eventos_por_pedal',
                              'cant_pedales_seg','duracion_seg','tracks_used','cant_grupos', 'info_tracks', 'midi_info'])
music_stats.loc[0] = [tonalidad, tonalidad_escala, scale_coverage,  mayor_chord_coverage, minor_chord_coverage, 
               tabla_esacla.iloc[0,1], tabla_esacla.iloc[1,1], tabla_esacla.iloc[2,1], 
               tabla_esacla.iloc[3,1], tabla_esacla.iloc[4,1], tabla_esacla.iloc[5,1], 
               tabla_esacla.iloc[6,1], avr_vertical_notes,cant_eventos_por_pedal,
               cant_pedales_seg,duracion_tema,n_tracks_used, cant_grupos, " ".join(info), mid]

data_describe = pd.DataFrame(tema_describe.loc[tema_describe.index != 'count',['note_num','Octave','start','duration']].unstack())
data_describe.reset_index(inplace=True)
data_describe.loc[:,'name'] = data_describe.level_0 + "_" + data_describe.level_1
data_describe.set_index('name',inplace=True)
data_describe.drop(['level_0','level_1'],axis=1, inplace=True)

pd.concat([music_stats.T,data_describe,velocity_avg,length_notes_avg])
    

noise notes: % 0.0
notes most used:


F5     10.47
A#5     8.71
G5      6.22
F4      6.04
C6      5.92
Name: note, dtype: float64

scale most used:


,index,note_name
0,F,23.34
1,A#,16.89
2,D,12.79
3,C,12.70
4,G,11.73
5,A,9.59
6,D#,9.33


mayor chord: 53.02 | minor chord: 41.41
A#M


,0
tonalidad,A#
tonalidad_escala,M
scale_coverage,96.37
mayor_chord_coverage,53.02
minor_chord_coverage,41.41
sc_1,23.34
sc_2,16.89
sc_3,12.79
sc_4,12.7
sc_5,11.73


In [18]:
mask_tiempo_from

1926    True
1928    True
1958    True
1962    True
1963    True
        ... 
2218    True
2219    True
2220    True
2302    True
1906    True
Name: segundo, Length: 3410, dtype: bool